<a href="https://colab.research.google.com/github/nirmal-ank/Transformers/blob/main/Masked_multi_headed_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.8 MB/s eta 0:00:00


In [2]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel
import torch
from bertviz import model_view
import pandas as pd
set_seed(42)

In [3]:
generator = pipeline('text-generation', model='gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [4]:
phrase = 'Learning about various transformers is very cool!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

`GPT2SdpaAttention` is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


12

In [5]:
encoded_phrase

{'input_ids': tensor([[41730,   546,  2972,  6121,   364,   318,   845,  3608,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
response.attentions[-1].shape  # From the final decoder batch size, head, tokens

torch.Size([1, 12, 9, 9])

In [7]:
encoded_phrase['input_ids'].shape


torch.Size([1, 9])

In [8]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

tokens

['Learning',
 'Ġabout',
 'Ġvarious',
 'Ġtransform',
 'ers',
 'Ġis',
 'Ġvery',
 'Ġcool',
 '!']

In [9]:
# Layer index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0] #tenth encode, first head, first batch

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df

<ipython-input-9-5f4da7d330dc>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)


,Learning,Ġabout,Ġvarious,Ġtransform,ers,Ġis,Ġvery,Ġcool,!
Learning,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.927,0.073,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġvarious,0.846,0.138,0.015,0.000,0.000,0.000,0.000,0.000,0.000
Ġtransform,0.905,0.036,0.005,0.054,0.000,0.000,0.000,0.000,0.000
ers,0.800,0.027,0.003,0.095,0.075,0.000,0.000,0.000,0.000
Ġis,0.688,0.117,0.007,0.003,0.002,0.182,0.000,0.000,0.000
Ġvery,0.342,0.128,0.014,0.002,0.001,0.490,0.022,0.000,0.000
Ġcool,0.849,0.064,0.002,0.002,0.001,0.051,0.007,0.025,0.000
!,0.660,0.048,0.003,0.018,0.046,0.022,0.006,0.022,0.175


In [10]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [11]:
response.hidden_states[-1].shape

torch.Size([1, 9, 768])

In [13]:
response.logits.shape

torch.Size([1, 9, 50257])

In [14]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,Learning,","
1,Ġabout,Ġthe
2,Ġvarious,Ġaspects
3,Ġtransform,ational
4,ers,Ġand
5,Ġis,Ġa
6,Ġvery,Ġimportant
7,Ġcool,.
8,!,Ċ


In [15]:
generator(phrase, max_length=30, num_return_sequences=1, do_sample=False)  # greedy search

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Learning about various transformers is very cool!\n\nI'm not sure if I'll be able to get the full version of the app, but"}]

In [16]:
generator(phrase, max_length=30, num_return_sequences=1, do_sample=True)  # greedy search with sampling


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Learning about various transformers is very cool!'}]